<a href="https://colab.research.google.com/github/cristiandarioortegayubro/accenture/blob/main/accenturedatascience03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![texto del vínculo](https://www.accenture.com/t20180820T081710Z__w__/il-en/_acnmedia/Accenture/Dev/Redesign/Acc_Logo_Black_Purple_RGB.PNG)

# Data Science / Datamining
~~~python
Cdor. Cristian Darío Ortega Yubro
~~~

De acuerdo al problema de negocios planteado, se quiere predecir cuando un empleado se va de la empresa, en virtud de las variables predictoras disponibles.

# Módulos

## Instalando módulos

In [ ]:
!pip install --upgrade plotly

## Para análisis de datos

In [ ]:
import pandas as pd
import numpy as np

## Para preprocesamiento y modelo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
import pickle #para grabar y recuperar los modelos entrenados
import os

## Para gráficos

In [ ]:
import plotly.express as px
import plotly.figure_factory as ff

##Obtención de datos y creación del dataframe

El conjunto de datos posee 10000 registros, de los cuales 8000 son de entrenamiento y 2000 son de testeo (80% - 20%), la hipotesis es que estos dos conjuntos de datos poseen igual distribución. 

El algoritmo seleccionado árbol de decisión es porque al correr el conjunto de datos train.csv con PyCaret, si bien no habían diferencias significativas en el accuracy entre los cinco mejores algoritmos que se ajustan a ese conjunto de datos, árbol de decisión tiene el mejor tiempo de procesamiento. Además, el árbol de decisión al tratarse de un método no paramétrico, no requiere que el conjunto de datos cumpla con algún tipo de distribución específica, como por ejemplo la distribución normal.

In [ ]:
train = "https://raw.githubusercontent.com/cristiandarioortegayubro/accenture/main/train.csv"
test = "https://raw.githubusercontent.com/cristiandarioortegayubro/accenture/main/test.csv"

In [ ]:
df_train = pd.read_csv(train)
df_train

,ID,nivel_de_satisfaccion,ultima_evaluacion,cantidad_proyectos,promedio_horas_mensuales_trabajadas,años_en_la_empresa,tuvo_un_accidente_laboral,promociones_ultimos_5_anios,area,salario,se_fue
0,2876.0,0.63,0.84,3,269,2,0,0,gestión de productos,bajo,no
1,7883.0,0.11,0.93,7,284,4,0,0,tecnica,bajo,si
2,4089.0,0.60,0.42,2,109,6,0,0,ventas,bajo,no
3,8828.0,0.38,0.49,4,196,3,0,1,dirección,alto,no
4,9401.0,0.11,0.83,6,244,4,0,0,contabilidad,bajo,si
...,...,...,...,...,...,...,...,...,...,...,...
7995,8701.0,0.63,0.85,2,156,3,1,0,RRHH,medio,no
7996,501.0,0.62,0.85,3,237,3,1,0,TI,medio,no
7997,2834.0,0.86,1.00,5,257,5,0,0,tecnica,medio,si
7998,8245.0,0.88,0.51,3,208,3,0,0,RRHH,medio,no


In [ ]:
df_train.drop(columns="ID", inplace=True)

In [ ]:
df_test = pd.read_csv(test)
df_test

,ID,nivel_de_satisfaccion,ultima_evaluacion,cantidad_proyectos,promedio_horas_mensuales_trabajadas,años_en_la_empresa,tuvo_un_accidente_laboral,promociones_ultimos_5_anios,area,salario,se_fue
0,2.0,0.40,0.57,2,152,3,0,0,ventas,bajo,NaN
1,4.0,0.54,0.52,3,115,3,0,0,contabilidad,bajo,NaN
2,9.0,0.72,0.87,5,223,5,0,0,ventas,bajo,NaN
3,14.0,0.65,0.67,3,245,3,0,0,ventas,medio,NaN
4,15.0,0.54,0.95,4,256,3,0,0,soporte,bajo,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1995,9980.0,0.57,0.85,4,164,3,0,0,ventas,bajo,NaN
1996,9987.0,0.78,0.98,5,239,6,0,0,marketing,bajo,NaN
1997,9991.0,0.09,0.93,6,279,4,0,0,tecnica,bajo,NaN
1998,9996.0,0.88,0.51,4,139,3,0,0,ventas,bajo,NaN


In [ ]:
df_test_pred = df_test.copy(deep=True)

In [ ]:
df_test.drop(columns="ID", inplace=True)

# Analisis exploratorio de los datos

## Resumen del tipo de datos

In [ ]:
df = pd.concat([df_test,df_train])
df.drop(columns="se_fue",inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   nivel_de_satisfaccion                10000 non-null  float64
 1   ultima_evaluacion                    10000 non-null  float64
 2   cantidad_proyectos                   10000 non-null  int64  
 3   promedio_horas_mensuales_trabajadas  10000 non-null  int64  
 4   años_en_la_empresa                   10000 non-null  int64  
 5   tuvo_un_accidente_laboral            10000 non-null  int64  
 6   promociones_ultimos_5_anios          10000 non-null  int64  
 7   area                                 10000 non-null  object 
 8   salario                              10000 non-null  object 
dtypes: float64(2), int64(5), object(2)
memory usage: 703.2+ KB


El dataframe no tiene valores nulos. De las nueve variables, dos de ellas no son numéricas. 

## Histogramas de las variables

Para visualizar la distribución de cada una de las variables

In [ ]:
df.columns

Index(['nivel_de_satisfaccion', 'ultima_evaluacion', 'cantidad_proyectos',
       'promedio_horas_mensuales_trabajadas', 'años_en_la_empresa',
       'tuvo_un_accidente_laboral', 'promociones_ultimos_5_anios', 'area',
       'salario'],
      dtype='object')

In [ ]:
for name in df.columns:
  fig=px.histogram(df, 
                   x=name, 
                   template="gridon", 
                   title="Histograma "+name, 
                   marginal="box")
  fig.update_layout(bargap=0.1)
  fig.show()

## Análisis descriptivo variables numéricas

In [ ]:
round(df.describe(),2).T

,count,mean,std,min,25%,50%,75%,max
nivel_de_satisfaccion,10000.0,0.61,0.25,0.09,0.44,0.64,0.82,1.0
ultima_evaluacion,10000.0,0.72,0.17,0.36,0.56,0.72,0.87,1.0
cantidad_proyectos,10000.0,3.80,1.24,2.00,3.00,4.00,5.00,7.0
promedio_horas_mensuales_trabajadas,10000.0,201.32,49.90,96.00,156.00,201.00,245.00,310.0
años_en_la_empresa,10000.0,3.49,1.47,2.00,3.00,3.00,4.00,10.0
tuvo_un_accidente_laboral,10000.0,0.15,0.36,0.00,0.00,0.00,0.00,1.0
promociones_ultimos_5_anios,10000.0,0.02,0.15,0.00,0.00,0.00,0.00,1.0


##Análisis descriptivo variables no numéricas

In [ ]:
df.select_dtypes(include=['object']).describe().T

,count,unique,top,freq
area,10000,10,ventas,2757
salario,10000,3,bajo,4863


## Matriz de correlación

In [ ]:
correlacion=round(df.corr(),2).values
nombres=list(df.corr().columns.values)
transposicion=correlacion[::-1]

In [ ]:
fig=ff.create_annotated_heatmap(transposicion, 
                                x=nombres,
                                y=nombres[::-1], 
                                colorscale='inferno')
fig.update_xaxes(side="bottom")

fig.show()

# División del conjunto de datos

In [ ]:
y_train = df_train["se_fue"]
y_test = df_test["se_fue"]

In [ ]:
X_train = df_train.drop(columns=["se_fue"])
X_test = df_test.drop(columns=["se_fue"])

#Preprocesamiento del conjunto de datos

##Variables categoricas y numericas

Se identifican las variables categóricas y las variables numéricas

In [ ]:
categoricas_train = X_train.select_dtypes(include=['object', 'category']).columns.to_list()
numericas_train = X_train.select_dtypes(include=['float64', 'int']).columns.to_list()

## Onehot para variables categoricas

Se aplica onehot solo para las variables categoricas, y se busca transformar todas las variables a números.

In [ ]:
preprocessor_train = ColumnTransformer(
                    [('onehot', OneHotEncoder(handle_unknown='ignore'), categoricas_train)],
                    remainder='passthrough')

Una vez que se ha definido el objeto ColumnTransformer, con el método fit()
se aprenden las transformaciones con los datos de entrenamiento y se aplican a
los conjuntos de datos con transform(). El resultado devuelto por ColumnTransformer es un array de numpy que no tiene los nombres de las columnas.

In [ ]:
X_train_prep = preprocessor_train.fit_transform(X_train)
X_test_prep = preprocessor_train.transform(X_test)

In [ ]:
encoded_cat_train = preprocessor_train.named_transformers_['onehot'].get_feature_names_out(categoricas_train)
labels_train = np.concatenate([numericas_train, encoded_cat_train])

Por defecto, OneHotEncoder ordena las nuevas columnas de izquierda a derecha por orden alfabético.

In [ ]:
X_train_prep = pd.DataFrame(X_train_prep, columns=labels_train)
X_test_prep = pd.DataFrame(X_test_prep, columns=labels_train)
X_train_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   nivel_de_satisfaccion                8000 non-null   float64
 1   ultima_evaluacion                    8000 non-null   float64
 2   cantidad_proyectos                   8000 non-null   float64
 3   promedio_horas_mensuales_trabajadas  8000 non-null   float64
 4   años_en_la_empresa                   8000 non-null   float64
 5   tuvo_un_accidente_laboral            8000 non-null   float64
 6   promociones_ultimos_5_anios          8000 non-null   float64
 7   area_ImásD                           8000 non-null   float64
 8   area_RRHH                            8000 non-null   float64
 9   area_TI                              8000 non-null   float64
 10  area_contabilidad                    8000 non-null   float64
 11  area_dirección                

Todas las variables son numéricas

# Creación del modelo Árbol de decisión

Se utiliza el algoritmo árbol de decisión definiendo variable respuesta se_fue y como predictores todas las variables disponibles. Se utilizan en primer lugar los hiperparámetros max_depth=5 y criterion='gini', el resto se dejan por defecto.

In [ ]:
modelo = DecisionTreeClassifier(max_depth=5, 
                                criterion="gini", 
                                random_state=2021)

In [ ]:
modelo.fit(X_train_prep, y_train)

DecisionTreeClassifier(max_depth=5, random_state=2021)

# Predicción

In [ ]:
predicciones = modelo.predict(X = X_test_prep,)
df_test_pred["prediccion"] = predicciones
df_test_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ID                                   2000 non-null   float64
 1   nivel_de_satisfaccion                2000 non-null   float64
 2   ultima_evaluacion                    2000 non-null   float64
 3   cantidad_proyectos                   2000 non-null   int64  
 4   promedio_horas_mensuales_trabajadas  2000 non-null   int64  
 5   años_en_la_empresa                   2000 non-null   int64  
 6   tuvo_un_accidente_laboral            2000 non-null   int64  
 7   promociones_ultimos_5_anios          2000 non-null   int64  
 8   area                                 2000 non-null   object 
 9   salario                              2000 non-null   object 
 10  se_fue                               0 non-null      float64
 11  prediccion                    

In [ ]:
df_test_pred.drop(columns=["nivel_de_satisfaccion","ultima_evaluacion","cantidad_proyectos",
                           "promedio_horas_mensuales_trabajadas","años_en_la_empresa",
                           "tuvo_un_accidente_laboral","promociones_ultimos_5_anios",
                           "area","salario","se_fue"],inplace=True)
df_test_pred.rename(columns={"prediccion":"se_fue"},inplace=True)

In [ ]:
df_test_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      2000 non-null   float64
 1   se_fue  2000 non-null   object 
dtypes: float64(1), object(1)
memory usage: 31.4+ KB


##Guardando archivo .csv

In [ ]:
df_test_pred.to_csv("submission03.csv")